### HASTA SAYISI BAR CHART

In [198]:
import plotly
import pandas as pd
#import plotly.plotly as py
import plotly.offline as pyo
from plotly.graph_objs import *
init_notebook_mode(connected=True)
import numpy as np
import matplotlib.pyplot as plt

In [286]:
def hasta_ve_saglikli_sayisi(user_df,df,temperature):
    df.set_index('UserId',inplace=True)
    df2=df.groupby(df.index).apply(lambda x: x.tail(40))## son 40 satır
    df3=df2.groupby('UserId').mean()
    df3['Hasta'] = 0
    df3['Saglikli'] = 0
    df3.loc[(df3['Temperature']>=temperature ) , 'Hasta'] = 1.0 ## Temperature degiscek
    df3.loc[(df3['Temperature']<temperature ) , 'Saglikli'] = 1.0 ## Temperature degiscek

    df3=df3.reset_index()
    
    df_merged = pd.merge(left=df3, right=user_df, how='left', left_on=('UserId'),right_on=('UserId')) ## ,how=left
    df4=df_merged.loc[:,['UserId','Age','Hasta','Saglikli','Temperature']]
    #df5=df4.groupby('Age').sum()
    
    df4['Rate'] = 0
    ### Temps
    df4.loc[(df4['Temperature']>38 ) , 'Rate'] = 1.0
    df4.loc[(df4['Temperature']>37.5) & (df4['Temperature']<38), 'Rate'] = 0.8
    df4.loc[(df4['Temperature']>37) & (df4['Temperature']<37.5), 'Rate'] = 0.6
    df4.loc[(df4['Temperature']>36.5) & (df4['Temperature']<37), 'Rate'] = 0.4
    df4.loc[(df4['Temperature']>36) & (df4['Temperature']<36.5), 'Rate'] = 0.2

    ### Ages
    df4.loc[(df4['Age']>=60),'Rate']+=0.6
    df4.loc[(df4['Age']<15,'Rate')]+=0.4
    df4.loc[(df4['Age']<60)&(df4['Age']>=50),'Rate']+=0.3
 
    df4['Risk']=0
    df4.loc[(df4['Rate']>=0.40 ) , 'Risk'] = 1
    
    df4.head()
    
    df_hastalikli=df4.loc[(df4['Hasta']>=1.0 )]
    df_saglikli=df4.loc[(df4['Saglikli']>=1.0 )]
    df_riskli=df4.loc[(df4['Risk']>=1.0 )]
    
  
    ##datayı 6 bolme?
    return df_hastalikli,df_saglikli,df_riskli
  


In [287]:
user_df=pd.read_csv("https://raw.githubusercontent.com/leventguner/kortsis/master/python_codes/user_df.csv?token=ANPQE2MLYNLNT52IZ33NE426X34DO")
df=pd.read_csv('https://raw.githubusercontent.com/leventguner/kortsis/master/python_codes/latest_df.csv?token=ANPQE2J2LQYJAGUWBEA35I26X7VAA')

user_df_=user_df.copy()
df_=df.copy()
df_hastalikli,df_saglikli,df_riskli=hasta_ve_saglikli_sayisi(user_df_,df_,temperature=36)

## Plotly

In [299]:
fig = go.Figure()


fig.add_trace(go.Histogram(
    x=df_saglikli['Age'],
     name='Saglikli',
     xbins=dict( # bins used for histogram
        start=0.0,
        end=70.0,
        size=10
    ),
    marker_color='indianred',
    opacity=0.75
))
fig.add_trace(go.Histogram(
    x=df_hastalikli['Age'],
     name='Hasta',
     xbins=dict( # bins used for histogram
        start=0.0,
        end=70.0,
        size=10
    ),
    marker_color='lightsalmon',
    opacity=0.75
))
fig.add_trace(go.Histogram(
    x=df_riskli['Age'],
    name='Riskli',
     xbins=dict( # bins used for histogram
        start=0.0,
        end=70.0,
        size=10
    ),
    opacity=0.75
))


fig.show()


### Ortalaması 5 yapsın,riskli=Sari

value_counts en yuksek olan kisinin id'sini gırıp tarhılerı alıp ona gore rısk puanı al

In [279]:
def Risk_Evde_Kalma(df):
    df3=df.groupby('ProcessDate').mean()
    df4=df3[(df3.index>'2020-05-08 00:30:00') ]
    np.random.seed(1)
    N=df4.shape[0]
    random_risk = np.random.randn(N)+5
    df5=df4.reset_index()
    df5['ProcessDate'] = pd.to_datetime(df5['ProcessDate'] )
    df6=df5.set_index('ProcessDate')
    df_risk=df6.resample('30T').mean() ## 60 sn bin
    
    df7=df5.reset_index()
    df7['Evde_Kalma_Orani'] = pd.DataFrame({'Evde_Kalma_Orani'  : np.random.randint(20, 30, size=df7.shape[0])})
    df7=df7.set_index('ProcessDate')
    df_evde_kalma=df7.resample('60T').mean() ## 60 sn bin
    
    return df_risk,df_evde_kalma



    

In [280]:
df=pd.read_csv('https://raw.githubusercontent.com/leventguner/kortsis/master/python_codes/latest_df.csv?token=ANPQE2J2LQYJAGUWBEA35I26X7VAA')
df_risk,df_evde_kalma=Risk_Evde_Kalma(df)

In [284]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_risk.index, y=random_risk,
                         marker_color='indianred',
                    mode='lines+markers',
                    name='lines+markers'))

fig.update_yaxes(range=[1,9])

fig.show()

In [285]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_evde_kalma.index, y=df_evde_kalma['Evde_Kalma_Orani'],
                         marker_color='indianred',
                    mode='lines+markers',
                    name='lines+markers'))

fig.update_yaxes(range=[20, 30],title_text="<b>Percantage</b>")

fig.show()